In [ ]:
import config
import pickle
import pipulate
import requests
from time import sleep
from sqlitedict import SqliteDict as sqldict
from bs4 import BeautifulSoup as bsoup

user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
headers = {"user-agent": user_agent}

# Build a list of all URLs found on homepage
clickdepthdb = f"{config.name}/clickdepths.db"
onedepth = []
with sqldict(clickdepthdb) as db:
    for url in db:
        depth = db[url]
        if depth == 1:
            onedepth.append(url)

In [ ]:
responsedb = f"{config.name}/responses.db"

for i, url in enumerate(onedepth):
    with sqldict(responsedb) as db:
        countdown = len(onedepth) - i
        if url in db:
            response = db[url]
            print(countdown, response.status_code, url)
        else:
            print(countdown, url, end=" ")
            response = requests.get(url, headers=headers)
            print(response.status_code)
            db[url] = response
            db.commit()
            sleep(.5)

In [ ]:
clickdepthdb = f"{config.name}/clickdepths.db"
edgesdb = f"{config.name}/edges.db"

with sqldict(responsedb) as db:
    for i, url in enumerate(onedepth):
        countdown = len(onedepth) - i
        print(countdown, end=" ")
        if url in db:
            response = db[url]
            if response.status_code == 200:
                soup = bsoup(response.text, "html.parser")
                links = pipulate.links(soup, url)
                # Record newly discoverd links as click-depth 2
                with sqldict(clickdepthdb) as db2:
                    for link in links:
                        if link not in db2:
                            db2[link] = 2
                    db2.commit()
                # Record what pages new links were found on (edges) 
                with sqldict(edgesdb) as db3:
                    for url in links:
                        atuple = (url, link)
                        db3[pickle.dumps(atuple)] = None
                    db3.commit()